In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Setup

In [5]:
data_dir = os.path.join(os.curdir, "data")

raw_filepath = os.path.join(data_dir, "clean-tweets.tsv")

In [6]:
raw = pd.read_csv(filepath_or_buffer=raw_filepath, sep="\t")

raw.head()

,Tweet,Country,Topic,Sentiment,Sentiment_Expression,Sentiment_Target,word_count,char_count,clean_text,clean_stemmed
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",lebanon,personal,negative,implicit,بريق العيون,23,132,أؤمن بأن الانسان ينطفئ جماله ابتعاد يحب بريق ا...,اؤم بأن انس طفئ جمل بعد يحب برق عين خفي صبح ذب...
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,jordan,sports,positive,explicit,افضل لاعب في العالم,23,141,الذاكره عندما اعتقد كريستيانو انه افضل لاعب ال...,ذكر عند عقد كريستيانو انه فضل لعب علم ككا يسي ...
2,لا نخلو من ضغوطات الحياة. فنحن نعيش على أرض أع...,palestine,personal,neutral,none,none,24,133,نخلو ضغوطات الحياة فنحن نعيش أرض أعدت للبلاء و...,خلو ضغط حية فنح نعش ارض اعد بلء ولم سلم بيء وك...
3,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,lebanon,personal,negative,explicit,مصطلحات_لبنانيه,23,135,بتوصل عالبيت بنط بقلك جيت بتقعد لتتحدث معو بقل...,وصل علب بنط بقل جيت قعد حدث معو بقل شو تقم تمش...
4,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,palestine,personal,negative,explicit,س لم تعد كما ك,16,67,نصمت لتسير حياتنا يرام فالناس تعد كآنت نقيه,نصم تسر حيت يرم لنس تعد كآن نقه


# Tokenization

First step is to split the dataset into small bits, each bit is called a `token`

once the corpus is tokenized, we can assign each unique `token` an `index`, note that the index value for any token is not important 

In [7]:
tweets = raw["clean_stemmed"]

In [8]:
sample = tweets.iloc[0]
sample

'اؤم بأن انس طفئ جمل بعد يحب برق عين خفي صبح ذبل طفئ تحل ربع خرف'

Tokenizing have many forms, examples: 
- Word Level Tokenization
- Character Level Tokenization
- Sub-word Tokenization
- Byte Pair Encoding

In this chapter we will cover word level and character level tokenization, others will be covered in later chapters

In [9]:
# Implement a generic Tokenizer class
# Different tokenizers will inherit from this class
class Tokenizer:
    def __init__(self, corpus: list[str]):
        self.vocab = self.create_vocab(corpus=corpus)
        
    def create_vocab(self, corpus: list[str]) -> dict[str, int]:
        ...
        
    def tokenize(self, documents: list[str]) -> list[list[int]]:
        ...

## Word Level

Here we split the sentence into indivual words, omitting the whitespaces between them

In [10]:
sample_tokens = sample.split(" ")
sample_tokens

['اؤم',
 'بأن',
 'انس',
 'طفئ',
 'جمل',
 'بعد',
 'يحب',
 'برق',
 'عين',
 'خفي',
 'صبح',
 'ذبل',
 'طفئ',
 'تحل',
 'ربع',
 'خرف']

In [11]:
all_tokens = []
for sample in tweets:
    for word in sample.split(" "):
        all_tokens.append(word)

In [12]:
len(all_tokens), len(set(all_tokens))

(61656, 7754)

In [13]:
# Create a list of all unique tokens in the corpus
word_level_tokens = [token for sample in tweets for token in sample.split(" ")]

word_level_tokens = set(word_level_tokens)
print(len(word_level_tokens))

7754


In [14]:
word2idx = {token: index for index, token in enumerate(word_level_tokens)}
word2idx

{'إرسال': 0,
 'شفو': 1,
 'قل': 2,
 'روي': 3,
 '😭💔': 4,
 'كيل”🌿🖤': 5,
 'تحط': 6,
 'كثر…': 7,
 'تبا': 8,
 'رسل…': 9,
 'بذن': 10,
 'ريكى': 11,
 'ببا': 12,
 'راة': 13,
 'كولش': 14,
 'فشل': 15,
 'طولكرم': 16,
 'كري': 17,
 'خله': 18,
 'ضحا': 19,
 'عفت': 20,
 'اس…': 21,
 'سرر': 22,
 'كنا': 23,
 'حبر': 24,
 'عنق': 25,
 'شهد': 26,
 'وتم': 27,
 'سبة': 28,
 'مره': 29,
 'ضار': 30,
 'ستع': 31,
 'فشئ': 32,
 'حتسب': 33,
 'وم…': 34,
 'اغا': 35,
 'هذاالفيلم': 36,
 'يند': 37,
 'بظل': 38,
 'وظف': 39,
 'صقع': 40,
 'حتمل…': 41,
 'شسع': 42,
 'فعر': 43,
 'ولل': 44,
 'اني': 45,
 'شمل': 46,
 'محمدقدم': 47,
 'بأخ': 48,
 'جسم': 49,
 'طبق': 50,
 'تخلف': 51,
 'رضه💀': 52,
 'وكس': 53,
 'قعة': 54,
 'شى': 55,
 'شقة': 56,
 'طمئن؛': 57,
 'خدو': 58,
 'سلع': 59,
 'ترث': 60,
 'تظل': 61,
 'أسمائ': 62,
 'خيط': 63,
 'سرع': 64,
 'ممتاز…': 65,
 'يثم': 66,
 'ﻻببض': 67,
 'إمآم': 68,
 'حرقص': 69,
 'عدي': 70,
 'جئتمو': 71,
 'بضح': 72,
 'زده': 73,
 'هالمصاري': 74,
 'رفج': 75,
 'تعط': 76,
 'حفز': 77,
 'خطراضيع': 78,
 'كأب': 79,
 'شرة

In [15]:
class WordLevelTokenizer(Tokenizer):
    ...

## Character Level Tokenization

In [16]:
sample_tokens = list(sample)
sample_tokens

['ر',
 'ق',
 'ب',
 ' ',
 'ن',
 'ق',
 'ل',
 ' ',
 'ب',
 'ش',
 'ر',
 ' ',
 'ن',
 'ش',
 'ر',
 ' ',
 'خ',
 'ب',
 'ر',
 ' ',
 'ر',
 'ي',
 'ض',
 ' ',
 'و',
 'ص',
 'ل',
 ' ',
 'ف',
 'ر',
 'ق',
 ' ',
 'ه',
 'و',
 'م',
 'ن',
 'ت',
 'م',
 ' ',
 'ب',
 'ط',
 'ل',
 ' ',
 'ع',
 'ر',
 'ب',
 ' ',
 'ن',
 'ط',
 'ق',
 ' ',
 'ب',
 'ر',
 'ج',
 ' ',
 'ح',
 'م',
 'د',
 ' ',
 '📍']

In [17]:
char_level_tokens = [token for tweet in tweets for token in tweet]

char_level_tokens = set(char_level_tokens)
print(len(char_level_tokens))

346


In [18]:
char2idx = {token: index for index, token in enumerate(char_level_tokens)}
char2idx

{'ﻟ': 0,
 'ج': 1,
 'ﺷ': 2,
 'س': 3,
 'ﺍ': 4,
 '🚦': 5,
 '😆': 6,
 '🏼': 7,
 '😟': 8,
 '⁰': 9,
 'ﻌ': 10,
 'ﺎ': 11,
 'ﺫ': 12,
 '😀': 13,
 '⚘': 14,
 '😂': 15,
 '💪': 16,
 'ี': 17,
 'م': 18,
 'ﻠ': 19,
 '💵': 20,
 '❓': 21,
 '״': 22,
 '🙂': 23,
 '❤': 24,
 'ا': 25,
 '😘': 26,
 '↷': 27,
 '✈': 28,
 '⠀': 29,
 '💡': 30,
 'ﻏ': 31,
 '☕': 32,
 '😑': 33,
 '✔': 34,
 '💀': 35,
 '♓': 36,
 '🇶': 37,
 '🍁': 38,
 '۞': 39,
 'ﺃ': 40,
 '😒': 41,
 '🍀': 42,
 '🙃': 43,
 '🤔': 44,
 'ﺸ': 45,
 '💴': 46,
 '🌿': 47,
 '￼': 48,
 '🏴': 49,
 '💶': 50,
 'ร': 51,
 '👆': 52,
 'ﺣ': 53,
 'ة': 54,
 'ﻔ': 55,
 '▪': 56,
 'ڪ': 57,
 '😐': 58,
 '🙆': 59,
 '🌚': 60,
 'ص': 61,
 '”': 62,
 '♥': 63,
 '🇷': 64,
 'ﺮ': 65,
 'ﻣ': 66,
 'ذ': 67,
 '😶': 68,
 '☺': 69,
 'ﻢ': 70,
 'ﻨ': 71,
 '⸀': 72,
 '💔': 73,
 '·': 74,
 '🏻': 75,
 '🕋': 76,
 'ﺬ': 77,
 '💞': 78,
 '🤷': 79,
 '😞': 80,
 '💖': 81,
 'ﷻ': 82,
 '💜': 83,
 'ﻃ': 84,
 'ﺟ': 85,
 '🎁': 86,
 'ﺨ': 87,
 '\u2069': 88,
 '»': 89,
 '🍃': 90,
 '💗': 91,
 '⭕': 92,
 '🏀': 93,
 '👮': 94,
 'ﻜ': 95,
 '😥': 96,
 'า': 97,
 '⚽': 98,
 '🌷': 99,
 'ٱ':

In [19]:
class CharacterLevelTokenizer(Tokenizer):
    ...

> Notice the difference in the vocabulary size, between word level and character level. Why would you choose one over the other? 

## Result of Tokenization

# n-grams

To be continued

# Save the Tokenizer

One of the most straight forward ways of saving a Python object is through binary `serialization`

serialization is a method of converting the `object` to `bytes`, these `bytes` can be read later to recreate the object

`pickle` package is the built-in package for object serialization